In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/798783/train_dataset_train.zip
!wget https://lodmedia.hb.bizmrg.com/case_files/798783/test_dataset_test.zip

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

In [ ]:
!pip install imblearn

In [ ]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import dbscan
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, RidgeClassifier, LogisticRegressionCV,Ridge,QuantileRegressor,PassiveAggressiveClassifier
from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier,RandomForestClassifier,VotingClassifier,RandomForestRegressor,GradientBoostingClassifier,StackingRegressor,BaggingClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import LinearSVC,LinearSVR,SVR
from sklearn.decomposition import TruncatedSVD,PCA,FactorAnalysis,IncrementalPCA,FastICA,KernelPCA,NMF
from sklearn.preprocessing import RobustScaler,QuantileTransformer,PowerTransformer,PolynomialFeatures,KBinsDiscretizer,StandardScaler,OneHotEncoder,OrdinalEncoder,FunctionTransformer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline,FeatureUnion,TransformerMixin
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,LocalOutlierFactor
from sklearn.model_selection import train_test_split,ShuffleSplit,StratifiedShuffleSplit,TimeSeriesSplit,GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn import set_config
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,roc_auc_score,accuracy_score,f1_score,classification_report
import re
from imblearn.pipeline import Pipeline as iPipeline
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
import pytesseract

In [ ]:
!unzip /content/train_dataset_train.zip
!unzip /content/test_dataset_test.zip

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Irkutsk/test.csv')
train=pd.read_csv('/content/drive/MyDrive/Irkutsk/train.csv')
train['date']=train['date'].astype('str').str.replace('2027','2022')
train['date']=pd.to_datetime(train['date'],errors='coerce')

In [ ]:
#парсим время в train1
vs = cv2.VideoCapture('/content/train/train1.avi')
slist,tlist=[],[]
for cnt in range(0,13194):
  vs.set(cv2.CAP_PROP_POS_MSEC,cnt*1000)
  ret, frame = vs.read()
  im=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[80:210,0:750]
  im=cv2.resize(im, (16*750,16*130))
  im=cv2.filter2D(im, -1, np.array([-1,5.5,-1]))
  im=cv2.resize(im, (750//2,130//2))
  slist.append(cnt)
  tlist.append(pytesseract.image_to_string(im))
train1_timing=pd.DataFrame({'Timing':slist,'Date_time':tlist})

In [ ]:
#парсим время в train2
vs = cv2.VideoCapture('/content/train/train2.avi')
slist,tlist=[],[]
for cnt in range(0,20034):
  vs.set(cv2.CAP_PROP_POS_MSEC,cnt*1000)
  ret, frame = vs.read()
  im=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[50:120,:560]
  im=cv2.resize(im, (16*560,16*70))
  im=cv2.filter2D(im, -1, np.array([-1,5.5,-1]))
  im=cv2.resize(im, (560//2,70//2))
  slist.append(cnt)
  tlist.append(pytesseract.image_to_string(im))
train2_timing=pd.DataFrame({'Timing':slist,'Date_time':tlist})

In [ ]:
#парсим время в test
vs = cv2.VideoCapture('/content/train/test.avi')
slist,tlist=[],[]
for cnt in range(0,12459):
  vs.set(cv2.CAP_PROP_POS_MSEC,cnt*1000)
  ret, frame = vs.read()
  im=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[50:120,:560]
  im=cv2.resize(im, (16*560,16*70))
  im=cv2.filter2D(im, -1, np.array([-1,5.5,-1]))
  im=cv2.resize(im, (560//2,70//2))
  slist.append(cnt)
  tlist.append(pytesseract.image_to_string(im))
test_timing=pd.DataFrame({'Timing':slist,'Date_time':tlist})

In [ ]:
#тайминг в трэйне
train_timing=pd.concat((train1_timing,train2_timing))
train_timing.index=range(len(train_timing))
train_timing['ind']=pd.Series(train_timing.index)
train_timing['hms1']=train_timing['Date_time'].str.extract(r"([0-2]\d:[0-5]\d:[0-5]\d)")
train_timing['hms2']=train_timing.loc[train_timing['hms1'].isna(),'Date_time'].str.extract(r"(\d:[0-5]\d:[0-5]\d)")
train_timing['hms']=(train_timing['hms1'].fillna('')+train_timing['hms2'].fillna(''))
train_timing['d']=train_timing['Date_time'].str.extract(r"(-2\d-)",expand=False).str.strip('-')
train_timing['d']=pd.to_numeric(train_timing['d'],errors='coerce')
dl=[train_timing['d'][0]]
for cnt in train_timing['d'][1:]:
  if cnt-dl[-1]==1:
    dl.append(cnt)
  else:
    dl.append(dl[-1])
train_timing['dy']=dl
train_timing['h']=train_timing['hms'].str.split(':').str[0]
train_timing['h']=pd.to_numeric(train_timing['h'],errors='coerce')
hl=[train_timing['h'][0],train_timing['h'][0]]
for cnt in zip(train_timing['h'][1:],train_timing['h'][1:].index):
  print(cnt[0],hl[-1],min(cnt[0],hl[-1]+1))
  if (cnt[0]>hl[-1] and cnt[0]-hl[-1]!=6) or hl[-1]-cnt[0]>9:
    hl.append(min(cnt[0],hl[-2]+1))
  else:
    hl.append(hl[-1])
train_timing['hr']=hl[1:]
train_timing['Date_time4']=pd.to_datetime(
    ('05-'+
    (train_timing['dy']).astype('int').astype('str')+
    '-2022 '+
    (train_timing['hr']+100).astype('int').astype('str').str[1:]+
    train_timing['hms'].str[-6:])
    ,errors='coerce')
train_timing['dh']=train_timing['Date_time4'].dt.day*24+train_timing['Date_time4'].dt.hour
hl=[train_timing['dh'][0]]
for cnt in train_timing['dh'][1:]:
  if cnt<=hl[-1] or cnt-hl[-1]>9:
    hl.append(hl[-1])
  else:
    hl.append(cnt)
train_timing['dh2']=hl
train_timing['dy2']=train_timing['dh2']//24
train_timing['hr2']=train_timing['dh2']%24
train_timing['Date_time5']=pd.to_datetime(
    ('05-'+
    (train_timing['dy2']).astype('int').astype('str')+
    '-2022 '+
    (train_timing['hr2']+100).astype('int').astype('str').str[1:]+
    train_timing['hms'].str[-6:])
    ,errors='coerce')
timing=pd.Series(train_timing.Timing)
timing.index=train_timing['Date_time5']
train3=pd.DataFrame({'indext':train.index})
train3.index=pd.to_datetime(train['date'],errors='coerce')
train3['Timing']=timing[~timing.index.duplicated()]
timing.index=timing.index+(timing.index[2]-timing.index[1])/2
train3['Timing+1']=timing[~timing.index.duplicated()]
timing.index=timing.index-(timing.index[2]-timing.index[1])
train3['Timing-1']=timing[~timing.index.duplicated()]
train3['cnt']=(1-train3['Timing+1'].isna())+(1-train3['Timing-1'].isna())
train3['timing_1']=(train3['Timing+1'].fillna(0)+train3['Timing-1'].fillna(0))/train3['cnt']
train3.loc[train3['Timing'].isna(),'Timing']=train3.loc[train3['Timing'].isna(),'timing_1']
train3=train.merge(train3,how='left',on='indext')
train3['File'].loc[:1269]='train1.avi'
train3['File'].loc[1269:]='train2.avi'


In [ ]:
#тайминг в тесте
test_timing['hms1']=test_timing['Date_time'].str.extract(r"([0-2]\d:[0-5]\d:[0-5]\d)")
test_timing['hms2']=test_timing.loc[test_timing['hms1'].isna(),'Date_time'].str.extract(r"(\d:[0-5]\d:[0-5]\d)")
test_timing['hms']=(test_timing['hms1'].fillna('')+test_timing['hms2'].fillna(''))
test_timing['d']=test_timing['Date_time'].str.extract(r"(-2\d-)",expand=False).str.strip('-')
test_timing['d']=pd.to_numeric(test_timing['d'],errors='coerce')
dl=[test_timing['d'][0]]
for cnt in test_timing['d'][1:]:
  if cnt-dl[-1]==1:
    dl.append(cnt)
  else:
    dl.append(dl[-1])
test_timing['dy']=dl
test_timing['h']=test_timing['hms'].str.split(':').str[0]
test_timing['h']=pd.to_numeric(test_timing['h'],errors='coerce')
hl=[test_timing['h'][0],test_timing['h'][0]]
for cnt in zip(test_timing['h'][1:],test_timing['h'][1:].index):
  print(cnt[0],hl[-1],min(cnt[0],hl[-1]+1))
  if (cnt[0]>hl[-1] and cnt[0]-hl[-1]!=6) or hl[-1]-cnt[0]>9:
    hl.append(min(cnt[0],hl[-2]+1))
  else:
    hl.append(hl[-1])
test_timing['hr']=hl[1:]
test_timing['Date_time4']=pd.to_datetime(
    ('05-'+
    (test_timing['dy']).astype('int').astype('str')+
    '-2022 '+
    (test_timing['hr']+100).astype('int').astype('str').str[1:]+
    test_timing['hms'].str[-6:])
    ,errors='coerce')
timing=pd.Series(test_timing.index)
timing.index=test_timing['Date_time4']
test2=pd.DataFrame({'indext':test.index})
test2.index=pd.to_datetime(test['date'],errors='coerce')
timing = timing[~timing.index.duplicated()]
test2['timing']=timing
timing.index=timing.index+(timing.index[2]-timing.index[1])/2
test2['timing+1']=timing
timing.index=timing.index-(timing.index[2]-timing.index[1])
test2['timing-1']=timing
test2['cnt']=(1-test2['timing+1'].isna())+(1-test2['timing-1'].isna())
test2['timing_1']=(test2['timing+1'].fillna(0)+test2['timing-1'].fillna(0))/test2['cnt']
test2.loc[test2['timing'].isna(),'timing']=test2.loc[test2['timing'].isna(),'timing_1']


In [ ]:
#обучающий пул кадров
train3['fname']=''
for cnt1 in ['train1.avi','train2.avi']:
  vs = cv2.VideoCapture('/content/train/'+cnt1)
  for cnt2 in train3[train3['File']==cnt1][train3[train3['File']==cnt1]['Timing'].isna()==False].index:
    vs.set(cv2.CAP_PROP_POS_MSEC,int(train3['Timing'][cnt2]*1000))
    ret, frame = vs.read()
    plt.imsave('/content/train3/'+str(cnt2)+'.png',frame)
    train3.loc[cnt2,['fname']]='/content/train3/'+str(cnt2)+'.png'

In [ ]:
#тестовый пул кадров
test2['fname']=''
vs = cv2.VideoCapture('/content/test/test.avi')
for cnt2 in test2.index[test2.timing.isna()==False]:
  vs.set(cv2.CAP_PROP_POS_MSEC,int(test2['timing'][cnt2]*1000))
  ret, frame = vs.read()
  plt.imsave('/content/test2/'+str(cnt2)+'.png',frame)
  test2.loc[cnt2,['fname']]='/content/test2/'+str(cnt2)+'.png'

In [ ]:
train2['employee']=train2['employee'].astype('str')
train2['action']=train2['action'].astype('str')
train2['date']=train2['date'].astype('str').str.replace('2027','2022')
train2['date']=pd.to_datetime(train2['date'],errors='coerce')
train2['1']=train2['date'].dt.dayofweek
train2['2']=train2['date'].dt.hour
train2['3']=train2['date'].dt.minute
train2['4']=train2['date'].dt.second

In [ ]:
test2['1']=test2['date'].dt.dayofweek
test2['2']=test2['date'].dt.hour
test2['3']=test2['date'].dt.minute
test2['4']=test2['date'].dt.second

In [ ]:
#обучение нейронных сетей
traingen = tf.keras.preprocessing.image.ImageDataGenerator(brightness_range=(0.9,1.1),channel_shift_range=0.1,
    preprocessing_function=tf.keras.applications.xception.preprocess_input)
traingen2 = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=(0.9,1.1),
    shear_range=0.1,
    zoom_range=0.1,
    channel_shift_range=0.0,
    preprocessing_function=tf.keras.applications.xception.preprocess_input)
valgen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.xception.preprocess_input)

In [ ]:
traingt=traingen.flow_from_dataframe(train2[train2['fname'].isna()==False], x_col='fname', y_col='action', 
                                    weight_col=None, target_size=(299, 299), 
                                    classes=list(train2.action.unique()),
                                    class_mode='categorical', batch_size=64)
traing2t=traingen2.flow_from_dataframe(train2[train2['fname'].isna()==False], x_col='fname', y_col='employee', 
                                    weight_col=None, target_size=(299, 299), 
                                    classes=list(train2.employee.unique()),
                                    class_mode='categorical', batch_size=64)

In [ ]:
inp=tf.keras.Input(shape=(299, 299, 3))

lay1=tf.keras.layers.Conv2D(8,(3,3),activation='relu')(inp)
lay2=tf.keras.layers.MaxPooling2D((5,5))(lay1)
lay3=tf.keras.layers.Conv2D(32,(3,3),activation='relu')(lay2)
lay4=tf.keras.layers.MaxPooling2D((2,2))(lay3)
lay5=tf.keras.layers.Flatten()(lay4)
lay6=tf.keras.layers.Dense(18,activation='softmax')(lay5)


model1 = tf.keras.Model(inputs=inp,outputs=lay6)
optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy',tf.keras.metrics.Recall()])

model1.summary()

In [ ]:
history=model1.fit(traingt,epochs=30)

In [ ]:
inp=tf.keras.Input(shape=(299, 299, 3))

lay1=tf.keras.layers.Conv2D(16,(3,3),activation='relu')(inp)
lay2=tf.keras.layers.MaxPooling2D((2,2))(lay1)
lay3=tf.keras.layers.Conv2D(32,(3,3),activation='relu')(lay2)
lay4=tf.keras.layers.MaxPooling2D((2,2))(lay3)
lay5=tf.keras.layers.Conv2D(64,(3,3),activation='relu')(lay4)
lay6=tf.keras.layers.MaxPooling2D((2,2))(lay5)
lay7=tf.keras.layers.Conv2D(128,(3,3),activation='relu')(lay6)
lay8=tf.keras.layers.MaxPooling2D((2,2))(lay7)
lay9=tf.keras.layers.Conv2D(256,(3,3),activation='relu')(lay8)
lay10=tf.keras.layers.GlobalMaxPooling2D()(lay9)
lay11=tf.keras.layers.Dense(64,activation='relu')(lay10)
lay12=tf.keras.layers.Dense(11,activation='softmax')(lay11)


model2 = tf.keras.Model(inputs=inp,outputs=lay12)
optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy',tf.keras.metrics.Recall()])

model2.summary()

In [ ]:
history=model2.fit(traing2t,epochs=30)

In [ ]:
X_data=valgen.flow_from_dataframe(train2[train2['fname'].isna()==False], x_col='fname', y_col='employee', 
                                    weight_col=None, target_size=(299, 299), 
                                    class_mode='categorical', batch_size=128,shuffle=False)
X_datat=valgen.flow_from_dataframe(test2[test2['fname'].isna()==False], x_col='fname', y_col=None, 
                                    weight_col=None, target_size=(299, 299), 
                                    class_mode=None, batch_size=128,shuffle=False)

In [ ]:
X1=model1.predict(X_data)
X1_1=np.zeros((len(train2),X1.shape[1]))
X1_1[train2['fname'].isna()==False]=X1

In [ ]:
X1t=model1.predict(X_datat)
X1t_1=np.zeros((len(test2),X1_1.shape[1]))
X1t_1[test2['fname'].isna()==False]=X1t

In [ ]:
X2=model2.predict(X_data)
X2_1=np.zeros((len(train2),X2.shape[1]))
X2_1[train2['fname'].isna()==False]=X2

In [ ]:
X2t=model2.predict(X_datat)
X2t_1=np.zeros((len(test2),X2_1.shape[1]))
X2t_1[test2['fname'].isna()==False]=X2t

In [ ]:
#конечный прогноз
pipe=iPipeline([
                ('im',SMOTETomek()),
                ('cls',RandomForestClassifier(200))])
test2['employee']=pipe.fit(np.hstack((X1_1,X2_1,train2[['1','2','3','4']].fillna(-1).values)),
                    train2['employee']).predict(np.hstack((X1t_1,X2t_1,test2[['1','2','3','4']].fillna(-1).values)))
test2['action']=pipe.fit(np.hstack((X1_1,X2_1,train2[['1','2','3','4']].fillna(-1).values))[train2['action']!='10.0'],
                    train2['action'][train2['action']!='10.0']).predict(np.hstack((X1t_1,X2t_1,test2[['1','2','3','4']].fillna(-1).values)))

In [ ]:
test2['employee']=test2['employee'].astype('int')
test2['action']=test2['action'].astype('float').astype('int')

In [ ]:
answer=test2[['id', 'employee', 'action']]
answer.to_csv('submit.csv',index=False)